In [1]:
import ROOT as rt
import csv
import re
import sys
import collections
import os
from collections import OrderedDict
import uproot
import pandas as pd

import scipy
import awkward
import numpy as np
import time
import numba
from numba import jit
from matplotlib import pyplot as plt
sys.path.append('/storage/af/user/christiw/login-1/christiw/DM/SNSPD_GaAs_limit/lib/')
from histo_utilities import create_TH1D, create_TH2D, std_color_list, create_TGraph


Welcome to JupyROOT 6.18/02


# load data from paper 1607.01009

In [2]:
path = '/storage/af/user/christiw/login-1/christiw/DM/SNSPD_GaAs_limit/data/'

ele_recoil = {}
limits = {}

ele_recoil['Fdm1_0p01GeV'] = np.genfromtxt(path + '1607.01009_Fig4_Fdm1_GaAs_0p01GeV.csv', delimiter = ',')
ele_recoil['Fdm1_1GeV'] = np.genfromtxt(path + '1607.01009_Fig4_Fdm1_GaAs_1GeV.csv', delimiter = ',')
ele_recoil['Fdmq-2_0p01GeV'] = np.genfromtxt(path + '1607.01009_Fig4_Fdmq-2_GaAs_0p01GeV.csv', delimiter = ',')
ele_recoil['Fdmq-2_1GeV'] = np.genfromtxt(path + '1607.01009_Fig4_Fdmq-2_GaAs_1GeV.csv', delimiter = ',')
xsec = 1e-37 #cm^2 assumption in paper for electron recoil histogram



limits['Fdmq-2_1ph'] = np.genfromtxt(path + '1607.01009_Fig2_limit_Fdmq-2_GaAs_1ph.csv', delimiter = ',')
limits['Fdm1_1ph'] = np.genfromtxt(path + '1607.01009_Fig2_limit_Fdm1_GaAs_1ph.csv', delimiter = ',')
limits['Fdmq-2_2ph'] = np.genfromtxt(path + '1607.01009_Fig2_limit_Fdmq-2_GaAs_2ph.csv', delimiter = ',')
limits['Fdm1_2ph'] = np.genfromtxt(path + '1607.01009_Fig2_limit_Fdm1_GaAs_2ph.csv', delimiter = ',')
limits['Fdmq-2_XENON'] = np.genfromtxt(path + '1607.01009_Fig2_limit_Fdmq-2_XENON10_1kg-year.csv', delimiter = ',')
limits['Fdm1_XENON'] = np.genfromtxt(path + '1607.01009_Fig2_limit_Fdm1_XENON10_1kg-year.csv', delimiter = ',')




# free parameters



In [6]:
### free parameters

############################################
### assumption used in paper 1607.01009
############################################
kg = 1
year = 1
light_yield = 1
LCE = 1
det_eff = 1
DCR = 0.0


############################################
### more realistic assumptions
############################################
# kg = 0.001
# year = 1./12
# light_yield = 0.2 #ph/eV
# LCE = 1
# det_eff = 0.01 
# DCR = 0.01 #cps

# signal and bkg yield

In [7]:
signal = {}
ph_num = {}
signal_yield_1ph = {}
signal_yield_2ph = {}
bkg_1ph = {}
bkg_2ph = {}
print("sample     \t signal(1ph)     \t signal(2ph)     \t bkg(1ph)     \t bkg(2ph)")
for k, v in ele_recoil.items():
    ph_num[k] = np.array(np.arange(1.5,len(v)+1, 1)) * light_yield
    signal[k] = v[:, 1]*kg * year * LCE * det_eff # events per eV
    
    
    # use this to reproduce limit from paper
    signal_yield_1ph[k] = np.sum(signal[k][ph_num[k]>=0])
    signal_yield_2ph[k] = np.sum(signal[k][ph_num[k]>= (6.5)])

#     signal_yield_1ph[k] = np.sum(signal[k][ph_num[k]>=1])
#     signal_yield_2ph[k] = np.sum(signal[k][ph_num[k]>=2])
#     bkg_1ph[k] = DCR * year * 3600 * 24 * 365.24
#     bkg_2ph[k] = DCR**2 * year * 3600 * 24 * 365.24


#     print(k, '\t', 1/signal_yield_1ph[k]*3.6*xsec, '\t', 1/signal_yield_2ph[k]*3.6*xsec, '\t', bkg_1ph[k], bkg_2ph[k])

    print(k, '\t', 1/signal_yield_1ph[k], '\t')
#     print(k, '\t', 1/signal_yield_1ph[k]*3.6*xsec, '\t', 1/signal_yield_2ph[k]*3.6*xsec)




sample     	 signal(1ph)     	 signal(2ph)     	 bkg(1ph)     	 bkg(2ph)
Fdm1_0p01GeV 	 4.0217921524736075e-05 	
Fdm1_1GeV 	 0.001096331412351862 	
Fdmq-2_0p01GeV 	 0.0002587870223916044 	
Fdmq-2_1GeV 	 0.013347360709403663 	
